# Renfe Scraping Software

This notebook is intended to show the process of scraping Renfe data using the `RenfeScraper` class. The class is designed to scrape data from the Renfe website and save it in a structured format. Two output files are obtained:
- `trips.csv`: Contains information about the trips, including the service ID, trip ID, and other relevant details.
- `stops.csv`: Contains information about the stops, including the stop ID, arrival and departure times, and other relevant details.

## 0. Import Libraries

In [3]:
%load_ext autoreload
%autoreload 2

import datetime

from robin.scraping.entities import DataLoader, SupplySaver
from robin.scraping.renfe.entities import RenfeScraper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Scrape available stations from main menu

In [4]:
scraper = RenfeScraper()

for station_id, station_name in scraper.available_stations.items():
    print(f'{station_id}: {station_name}')

31412: A Coruña
94707: Abrantes
60911: Alicante / Alacant
60600: Albacete
06008: Alcantarilla-Los Romanos
60400: Alcázar de San Juan
55020: Algeciras
56312: Almería
99003: Altet Bus
99115: Aguadulce Bus
87912: Aix En Provence
99114: Andorra-Bus
ANTEQ: Antequera (TODAS)
87814: Avignon
10400: Avila
37606: Badajoz
BARCE: Barcelona (TODAS)
87078: Beziers
65318: Benicassim
BILBA: Bilbao (TODAS)
54400: Bobadilla
11014: Burgos Rosa Manzano
35400: Cáceres
51405: Cádiz
70600: Calatayud
50417: Campus Rabanales
61307: Cartagena
65300: Castellón /Castelló
37200: Ciudad Real
50500: Córdoba
CUENC: Cuenca (TODAS)
92201: Denia-Bus
60905: Elda-Petrer
03410: Elche AV/Elx AV
94428: Entroncamento
92157: Estepona Bus
21010: Ferrol
79309: Figueres
79333: Figueres Bus
04307: Figueres Vilafant
69110: Gandía
GIJON: Gijón
79300: Girona
05000: Granada
GUADA: Guadalajara (TODAS)
43019: Huelva
74200: Huesca
IRUN-: Irun-Hendaya (TODAS)
80100: Pamplona/Iruña
99103: Jaca-Bus
03100: Jaén
64100: Xàtiva/Játiva
97639: Ja

In [5]:
scraper.stations_df

,ADIF_ID,RENFE_ID,STATION_NAME,LATITUD,LONGITUD,DIRECION,CP,POBLACION,PROVINCIA,PAIS,CERCANIAS,FEVE
0,01001,01001,EL SORBITO (APD-CGD),37.208475,-5.706642,NaN,NaN,ALCALÁ DE GUADAÍRA,SEVILLA,ESPAÑA,NO,NO
1,01002,01002,LA TRINIDAD (APT-CGD),NaN,NaN,NaN,NaN,ALCALÁ DE GUADAÍRA,SEVILLA,ESPAÑA,NO,NO
2,01003,01003,ARAHAL,37.268081,-5.548514,"Calle Virgen de los Dolores, S/N",41600,ARAHAL,SEVILLA,ESPAÑA,NO,NO
3,01004,01004,PARADAS (APD-CGD),NaN,NaN,NaN,NaN,PARADAS,SEVILLA,ESPAÑA,NO,NO
4,01005,01005,MARCHENA,37.334282,-5.425519,"Avenida Maestro Santos Ruano, 8",41620,MARCHENA,SEVILLA,ESPAÑA,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
2772,99501,99501,ANDORRA,NaN,NaN,NaN,NaN,ANDORRA,PONTEVEDRA,ESPAÑA,NO,NO
2773,99800,99800,CERCEDILLA TURÍSTICO,NaN,NaN,NaN,NaN,CERCEDILLA,MADRID,ESPAÑA,SI,NO
2774,99801,99801,PUERTO NAVACERRADA,NaN,NaN,NaN,NaN,NAVACERRADA,MADRID,ESPAÑA,NO,NO
2775,99802,99802,COTOS,NaN,NaN,NaN,NaN,RASCAFRÍA,MADRID,ESPAÑA,NO,NO


## 2. Scrape Renfe services

The following cell scrapes the Renfe services for a specific date and range of days. The `origin` and `destination` variables specify the departure and arrival stations, respectively. The `day`, `month`, and `year` variables specify the date for which the services are to be scraped. The `range_days` variable specifies the number of days to scrape.

In [6]:
origin = '60000'
destination = '03216'

day=5
month=6
year=2025
range_days=1

date = datetime.date(day=day, month=month, year=year)
scraper.scrape(
    origin=origin,
    destination=destination,
    init_date=date,
    range_days=1,
    all_pairs=False,
    save_path='../data/renfe/2025/'
)

2025-04-08 18:58:59.882 | INFO     | robin.scraping.renfe.entities:scrape_trips:768 - Scraping trips for MADRI - VALEN on 2025-06-05
2025-04-08 18:58:59.882 | INFO     | robin.scraping.renfe.entities:_get_renfe_schedules_url:323 - https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=MADRI&D=VALEN&AF=2025&MF=06&DF=05&SF=4&ID=s


2025-04-08 18:59:08.460 | SUCCESS  | robin.scraping.renfe.entities:scrape:673 - Scraped 67 trips between MADRI and VALEN from 2025-06-05 to 2025-06-06
2025-04-08 18:59:08.463 | INFO     | robin.scraping.renfe.entities:scrape:674 - First five trips:
               service_id stop_id  arrival  departure
0  05064_05-06-2025-06.30   17000        0          0
1  05064_05-06-2025-06.30   03208       63         65
2  05064_05-06-2025-06.30   03213       98         99
3  05064_05-06-2025-06.30   03216      124        124
4  05070_05-06-2025-07.30   17000        0          0
2025-04-08 18:59:08.465 | INFO     | robin.scraping.renfe.entities:scrape_prices:726 - Scraping prices for MADRI - VALEN on 2025-06-05
2025-04-08 18:59:08.467 | INFO     | robin.scraping.renfe.entities:_get_renfe_prices_url:305 - https://venta.renfe.com/vol/buscarTren.do?tipoBusqueda=autocomplete&currenLocation=menuBusqueda&vengoderenfecom=SI&cdgoOrigen=MADRI&cdgoDestino=VALEN&idiomaBusqueda=s&FechaIdaSel=05/06/2025&_fechaI

## 3. Load scraped data

The following cell loads the scraped data from the `trips.csv` and `stops.csv` files. The `DataLoader` class is used to load the data and create a supply entity.

In [7]:
# The different seats with their hard and soft types
seat_components = {
    'Básica': (1, 1),
    'Básico': (1, 1),
    'Elige': (1, 2),
    'Elige Confort': (1, 3),
    'Prémium': (2, 4)
}

# The number of seats available for each hard type
seat_quantity = {
    1: 250,
    2: 50,
}

data_loader = DataLoader(
    stops_path='../data/renfe/2025/stop_times/stopTimes_MADRI_BARCE_2025-06-05_2025-06-06.csv',
    prices_path='../data/renfe/2025/prices/prices_MADRI_BARCE_2025-06-05_2025-06-06.csv',
    seat_components=seat_components,
    seat_quantity=seat_quantity
)

In [8]:
for service in data_loader.services:
    print(service)

Service id: 06301_05-06-2025-06.12 
	Date of service: 2025-06-05 
	Stops: ['60000', '71801'] 
	Line times (relative): [(0, 0), (157, 157)] 
	Line times (absolute): [('06:12', '06:12'), ('08:49', '08:49')] 
	Train Service Provider: Renfe 
	Time Slot: 37210 
	Rolling Stock: S-114 
	Prices: 
		('60000', '71801'): {Básica: 25.0} 
	Tickets sold (seats): 
		Básica: 0 
	Tickets sold (hard type): 
		1: 0
		2: 0 
	Tickets sold per each pair (seats): 
		('60000', '71801'): {Básica: 0} 
	Capacity constraints: None 

Service id: 03063_05-06-2025-06.27 
	Date of service: 2025-06-05 
	Stops: ['60000', '70600', '04040', '71801'] 
	Line times (relative): [(0, 0), (59, 60), (85, 86), (178, 178)] 
	Line times (absolute): [('06:27', '06:27'), ('07:26', '07:27'), ('07:52', '07:53'), ('09:25', '09:25')] 
	Train Service Provider: Renfe 
	Time Slot: 38710 
	Rolling Stock: S-114 
	Prices: 
		('60000', '71801'): {Básico: 28.75}
		('60000', '71801'): {Elige: 40.2}
		('60000', '71801'): {Elige Confort: 53.5}
		(

## 4. Save supply entities

The following cell saves the supply entities to a YAML file. The `SupplySaver` class is used to save the data in a structured format. The `to_yaml` method is used to save the data to a file.

In [9]:
supply_saver = SupplySaver(data_loader.services)
supply_saver.to_yaml(output_path='../data/renfe/2025/supply_2025_06_06_MADRI_BARCE.yaml')